In [2]:
# start coding here
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc

%matplotlib inline

sns.set_style('ticks')
sns.set_context('talk')

In [3]:
read_counts = pd.read_csv(snakemake.input[0])
samp2mapped = read_counts.groupby(['sample_name'])['Coverage'].sum().to_dict()
read_counts['fraction_reads'] = read_counts.apply(
    lambda x: x.Coverage/samp2mapped[x.sample_name],
    axis=1)
read_counts.head()

In [4]:
read_counts.sample_name.unique()

In [5]:
sample_df = pd.read_csv(snakemake.config['samples'], sep='\t')
sample_df.sort_values(by=['timepoint',], inplace=True)
sample_order = sample_df.sample_name.values
sample_df.head()

In [6]:
t0 = sample_df.timepoint.min()
t0_samples = sample_df[sample_df.timepoint == t0].sample_name.values
t0_samples

In [7]:
pal = sns.color_palette('colorblind', len(t0_samples))
fig, axes = plt.subplots(figsize=(13, 5), ncols=len(t0_samples))
for smpl, ax in zip(t0_samples, axes):
    y = read_counts[read_counts.sample_name == smpl].fraction_reads.sort_values(
        ascending=False).values
    x = np.arange(0, len(y)) / len(y)
    y = np.cumsum(y)
    ax.plot(x, y, c=pal.pop(0))
    ax.plot([0, x[-1]], [0, y[-1]], '--', c='grey')
    lbl = auc(x, y)
    ax.text(0, 0.9, "AUC = {:.3f}".format(lbl))
    ax.set_title(smpl)
    ax.set_xlabel("Ranked Guides")
    if smpl == t0_samples[0]:
        ax.set_ylabel("Fraction Reads")
plt.savefig(snakemake.output[0])
prefix, _ = os.path.splitext(snakemake.output[0])
plt.savefig(prefix + '.png')

In [8]:
g = sns.FacetGrid(read_counts,
                  row='sample_name',
                  row_order=t0_samples,
                  hue='sample_name',
                  hue_order=t0_samples,
                  sharex=False,
                  sharey=False,
                  height=6,
                  aspect=1.5)
g.map_dataframe(sns.histplot, 'Coverage')
plt.savefig(snakemake.output[1])
prefix, _ = os.path.splitext(snakemake.output[1])
plt.savefig(prefix + '.png')

In [9]:
g = sns.FacetGrid(read_counts,
                  row='sample_name',
                  row_order=sample_order,
                  hue='sample_name',
                  hue_order=sample_order,
                  sharex=False,
                  sharey=False,
                  height=6,
                  aspect=1.5)
g.map_dataframe(sns.histplot, 'Coverage')
plt.savefig(snakemake.output[2])
prefix, _ = os.path.splitext(snakemake.output[2])
plt.savefig(prefix + '.png')